In [1]:
import torch
import polars as pl
from transformers import AutoTokenizer, AutoModelForSequenceClassification
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

LABEL_NAMES = ["ENTAILMENT", "NEUTRAL", "CONTRADICTION"]

df = pl.read_csv("data/gpt4o_raw_03.06.2024.csv")
print("Total samples", df.height)
df.columns

Total samples 10


['cid',
 'premise',
 'hypothesis',
 'label',
 'Generated ENTAILMENT Hypothesis',
 'ENTAILMENT Correct?',
 'ENTAILMENT Adversarial?',
 'Generated NEUTRAL Hypothesis',
 'NEUTRAL Correct?',
 'NEUTRAL Adversarial',
 'Generated CONTRADICTION Hypothesis',
 'CONTRADICTION Correct?',
 'CONTRADICTION Adversarial']

# Compute the scores
Check whether the samples generated by LLama3 that are *correct* (i.e., checked by a human), are also *difficult* for the model (i.e., the model fails to classify them correctly).

This section only writes on another `.csv` file the results.

In [2]:
MODELS = {}
TOKENIZERS = {}

model_name_base = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
model_name_large = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
model_name_large_2 = "Joelzhang/deberta-v3-large-snli_mnli_fever_anli_R1_R2_R3-nli"
MAP = {
    'base' : model_name_base,
    'large1': model_name_large,
    'large2': model_name_large_2,
}

for model_name in MAP.values():
    print(f"> loading {model_name}")
    TOKENIZERS[model_name] = AutoTokenizer.from_pretrained(model_name, cache_dir='./.hf_cache')
    MODELS[model_name] = AutoModelForSequenceClassification.from_pretrained(model_name, cache_dir='./.hf_cache').to(device)


def inference(model_name, premise, hypothesis):
    model_input = TOKENIZERS[model_name](premise, hypothesis, truncation=False, return_tensors="pt")
    output = MODELS[model_name](model_input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
    prediction = torch.softmax(output["logits"][0], -1).tolist()
    return {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, LABEL_NAMES)}

def predict(model_name, premise, hypothesis, label):
    prediction = inference(model_name, premise, hypothesis)
    predicted = max(prediction, key=prediction.get)
    return int(predicted != label)

> loading MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli
> loading MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli
> loading Joelzhang/deberta-v3-large-snli_mnli_fever_anli_R1_R2_R3-nli


/media/ssd/gioffre/langchain/.conda/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
from tqdm import tqdm
from pprint import pprint
res = {}
for label in LABEL_NAMES:
    for model_id in MAP.keys():
        res[f"new {label} hypothesis fools {model_id.upper()}"] = []
#pprint(res)

for model_id, model_name in MAP.items():
    for i, elem in tqdm(enumerate(df.iter_rows(named=True)), desc=f'> model {model_id}', total=df.height):
        for label in LABEL_NAMES:
            if elem[f'{label} Correct?'] == True:
                int_flag = predict(model_name, elem['premise'], elem['hypothesis'], elem['label'])
                res[f'new {label} hypothesis fools {model_id.upper()}'].append(int_flag)
            else:
                res[f'new {label} hypothesis fools {model_id.upper()}'].append(None)
#pprint(res)

> model large2: 100%|██████████| 10/10 [00:00<00:00, 28.80it/s]


In [4]:
for label in LABEL_NAMES:
    for model_id in MAP.keys():
        df = df.with_columns(
            pl.Series(
                name=f"new {label} hypothesis fools {model_id.upper()}", 
                values=res[f"new {label} hypothesis fools {model_id.upper()}"]
            ),
        )
df.columns
df.head()

cid,premise,hypothesis,label,Generated ENTAILMENT Hypothesis,ENTAILMENT Correct?,ENTAILMENT Adversarial?,Generated NEUTRAL Hypothesis,NEUTRAL Correct?,NEUTRAL Adversarial,Generated CONTRADICTION Hypothesis,CONTRADICTION Correct?,CONTRADICTION Adversarial,new ENTAILMENT hypothesis fools BASE,new ENTAILMENT hypothesis fools LARGE1,new ENTAILMENT hypothesis fools LARGE2,new NEUTRAL hypothesis fools BASE,new NEUTRAL hypothesis fools LARGE1,new NEUTRAL hypothesis fools LARGE2,new CONTRADICTION hypothesis fools BASE,new CONTRADICTION hypothesis fools LARGE1,new CONTRADICTION hypothesis fools LARGE2
i64,str,str,str,str,bool,bool,str,bool,bool,str,bool,bool,i64,i64,i64,i64,i64,i64,i64,i64,i64
184052,"""Gangs of New York . The screen…","""Kenneth Lonergan is a writer o…","""ENTAILMENT""","""Kenneth Lonergan, who contribu…",true,false,"""Kenneth Lonergan, known for hi…",true,false,"""Gangs of New York was co-writt…",true,false,0,0,0,0,0,0,0,0,0
217178,"""Pakistan Movement . The Pakist…","""The Pakistan Movement was the …","""NEUTRAL""","""The Pakistan Movement involved…",true,false,"""The Pakistan Movement was prim…",true,false,"""The Pakistan Movement had no i…",true,false,0,0,0,0,0,0,0,0,0
169984,"""The Brat Pack is a nickname gi…","""The Brat Pack is a nickname gi…","""CONTRADICTION""","""The Brat Pack consisted of act…",true,false,"""Members of the Brat Pack were …",true,false,"""The Brat Pack refers to a grou…",true,false,1,1,1,1,1,1,1,1,1
120315,"""Bret Easton Ellis . Ellis also…","""Bret Easton Ellis barely wrote…","""CONTRADICTION""","""Bret Easton Ellis is the write…",true,false,"""Bret Easton Ellis's work on Th…",true,false,"""The Canyons, a 2013 film, was …",true,false,1,1,1,1,1,1,1,1,1
145735,"""New York Knicks . The Knicks c…","""The New York Knicks are in the…","""ENTAILMENT""","""The New York Knicks are part o…",true,false,"""The New York Knicks have won m…",true,false,"""The New York Knicks are a foot…",true,false,0,0,0,0,0,0,0,0,0


In [5]:
new_df = df.with_columns(
    (
        pl.col("new ENTAILMENT hypothesis fools BASE")   +  
        pl.col("new ENTAILMENT hypothesis fools LARGE1") + 
        pl.col("new ENTAILMENT hypothesis fools LARGE2")
    ).alias("ENTAILMENT difficulty score"),
    (
        pl.col("new NEUTRAL hypothesis fools BASE")   +  
        pl.col("new NEUTRAL hypothesis fools LARGE1") + 
        pl.col("new NEUTRAL hypothesis fools LARGE2")
    ).alias("NEUTRAL difficulty score"),
    (
        pl.col("new CONTRADICTION hypothesis fools BASE")   +  
        pl.col("new CONTRADICTION hypothesis fools LARGE1") + 
        pl.col("new CONTRADICTION hypothesis fools LARGE2")
    ).alias("CONTRADICTION difficulty score"),
)
new_df.head()

cid,premise,hypothesis,label,Generated ENTAILMENT Hypothesis,ENTAILMENT Correct?,ENTAILMENT Adversarial?,Generated NEUTRAL Hypothesis,NEUTRAL Correct?,NEUTRAL Adversarial,Generated CONTRADICTION Hypothesis,CONTRADICTION Correct?,CONTRADICTION Adversarial,new ENTAILMENT hypothesis fools BASE,new ENTAILMENT hypothesis fools LARGE1,new ENTAILMENT hypothesis fools LARGE2,new NEUTRAL hypothesis fools BASE,new NEUTRAL hypothesis fools LARGE1,new NEUTRAL hypothesis fools LARGE2,new CONTRADICTION hypothesis fools BASE,new CONTRADICTION hypothesis fools LARGE1,new CONTRADICTION hypothesis fools LARGE2,ENTAILMENT difficulty score,NEUTRAL difficulty score,CONTRADICTION difficulty score
i64,str,str,str,str,bool,bool,str,bool,bool,str,bool,bool,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
184052,"""Gangs of New York . The screen…","""Kenneth Lonergan is a writer o…","""ENTAILMENT""","""Kenneth Lonergan, who contribu…",true,false,"""Kenneth Lonergan, known for hi…",true,false,"""Gangs of New York was co-writt…",true,false,0,0,0,0,0,0,0,0,0,0,0,0
217178,"""Pakistan Movement . The Pakist…","""The Pakistan Movement was the …","""NEUTRAL""","""The Pakistan Movement involved…",true,false,"""The Pakistan Movement was prim…",true,false,"""The Pakistan Movement had no i…",true,false,0,0,0,0,0,0,0,0,0,0,0,0
169984,"""The Brat Pack is a nickname gi…","""The Brat Pack is a nickname gi…","""CONTRADICTION""","""The Brat Pack consisted of act…",true,false,"""Members of the Brat Pack were …",true,false,"""The Brat Pack refers to a grou…",true,false,1,1,1,1,1,1,1,1,1,3,3,3
120315,"""Bret Easton Ellis . Ellis also…","""Bret Easton Ellis barely wrote…","""CONTRADICTION""","""Bret Easton Ellis is the write…",true,false,"""Bret Easton Ellis's work on Th…",true,false,"""The Canyons, a 2013 film, was …",true,false,1,1,1,1,1,1,1,1,1,3,3,3
145735,"""New York Knicks . The Knicks c…","""The New York Knicks are in the…","""ENTAILMENT""","""The New York Knicks are part o…",true,false,"""The New York Knicks have won m…",true,false,"""The New York Knicks are a foot…",true,false,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
new_df.write_csv("data/gpt4o_raw_03.06.2024_STATS.csv")
print("> Written stats csv.")

> Written stats csv.


# Statistics
I want to see:
- how many generated examples are correct for each category
- how many correct examples are can fool 0, 1, 2, 3 models

In [10]:
from rich.console import Console
from rich.table import Table

console = Console()

new_df = pl.read_csv("data/gpt4o_raw_03.06.2024_STATS.csv")

In [11]:
table = Table(title='gpt4o_generated_03.06.2024_STATS.csv')

table.add_column("Category", justify="left", style="cyan", no_wrap=True)
table.add_column("Percentage", style="green", justify='center')
table.add_column("Values", justify="right", style="magenta")

for label in LABEL_NAMES:
    num_correct = new_df.filter(pl.col(f'{label} Correct?') == True).height
    table.add_row(
        f"Synthetic {label:<13} samples that are correct:", 
        f"{100*num_correct/new_df.height:.2f}%", 
        f"{num_correct}/{new_df.height}") 

console.print(table)
          

                   gpt4o_generated_03.06.2024_STATS.csv                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ Category                                          ┃ Percentage ┃ Values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ Synthetic ENTAILMENT    samples that are correct: │  100.00%   │  10/10 │
│ Synthetic NEUTRAL       samples that are correct: │  100.00%   │  10/10 │
│ Synthetic CONTRADICTION samples that are correct: │  100.00%   │  10/10 │
└───────────────────────────────────────────────────┴────────────┴────────┘

In [12]:
table = Table(title='gpt4o_generated_03.06.2024_STATS.csv')

table.add_column("Category of correct synthetic samples", justify="left", style="cyan", no_wrap=False)
table.add_column("Fool 0 models", style="red", justify='center')
table.add_column("%", style="red", justify='center')
table.add_column("Fool >= 1 models", style="green", justify='center')
table.add_column("%", style="green", justify='center')
table.add_column("Fool >= 2 models", style="green", justify='center')
table.add_column("%", style="green", justify='center')
table.add_column("Fool >= 3 models", style="green", justify='center')
table.add_column("%", style="green", justify='center')
table.add_column("Support", justify="right", style="magenta")

for label in LABEL_NAMES:
    # retrieve correct samples per label
    correct_df = new_df.filter(pl.col(f'{label} Correct?') == True)
    num_correct = correct_df.height

    fool_0 = correct_df.filter(pl.col(f"{label} difficulty score") == 0).height
    fool_at_least_1 = correct_df.filter(pl.col(f"{label} difficulty score") >= 1).height
    fool_at_least_2 = correct_df.filter(pl.col(f"{label} difficulty score") >= 2).height
    fool_at_least_3 = correct_df.filter(pl.col(f"{label} difficulty score") >= 3).height

    table.add_row(
        label, 
        f"{fool_0:>2}",
        f"{100*fool_0/num_correct:.2f}%", 
        f"{fool_at_least_1:>2}",
        f"{100*fool_at_least_1/num_correct:.2f}%", 
        f"{fool_at_least_2:>2}",
        f"{100*fool_at_least_2/num_correct:.2f}%", 
        f"{fool_at_least_3:>2}",
        f"{100*fool_at_least_3/num_correct:.2f}%", 
        f"{num_correct}"
) 

console.print(table)

                                       gpt4o_generated_03.06.2024_STATS.csv                                        
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓
┃ Category of ┃             ┃        ┃            ┃        ┃             ┃        ┃            ┃        ┃         ┃
┃ correct     ┃             ┃        ┃            ┃        ┃             ┃        ┃            ┃        ┃         ┃
┃ synthetic   ┃   Fool 0    ┃        ┃ Fool >= 1  ┃        ┃  Fool >= 2  ┃        ┃ Fool >= 3  ┃        ┃         ┃
┃ samples     ┃   models    ┃   %    ┃   models   ┃   %    ┃   models    ┃   %    ┃   models   ┃   %    ┃ Support ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩
│ ENTAILMENT  │      7      │ 70.00% │      3     │ 30.00% │      3      │ 30.00% │      3     │ 30.00% │      10 │
│ NEUTRAL     │      7      │ 70.00% │      3     │ 30.00% │      3      │ 30.00% │      3     │ 30.00% │      10 │
│ CONTRADICT… │      7      │ 70.00% │      3     │ 30.00% │      3      │ 30.00% │      3     │ 30.00% │      10 │
└─────────────┴─────────────┴────────┴────────────┴────────┴─────────────┴────────┴────────────┴────────┴─────────┘